### codebase taken from:

https://www.tensorflow.org/tutorials/text/text_generation

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [2]:
tf.version.VERSION

'2.4.1'

In [3]:
path_to_file = 'Huxley_-_Brave_New_World.txt'

### Read the data

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='ISO-8859-1')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 380264 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])


    

      Aldous Huxley (1894-1963)
      Brave New World (1932)
    

    

      

        Chapter One
      

      A squat grey building of only thirty-four stories. Over the main entrance the words, CENTRAL LONDON HATCHERY AND C


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

82 unique characters


### Process the text

In [7]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))

In [8]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [9]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Create training examples and targets

In [10]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(380264,), dtype=int64, numpy=array([ 3,  2,  4, ..., 56,  3,  2])>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
seq_length = 25
examples_per_epoch = len(text)//(seq_length+1)

In [13]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'\r' b'\n' b' ' b' ' b' ' b' ' b'\r' b'\n' b'\r' b'\n' b' ' b' ' b' '
 b' ' b' ' b' ' b'A' b'l' b'd' b'o' b'u' b's' b' ' b'H' b'u' b'x'], shape=(26,), dtype=string)


In [14]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'\r\n    \r\n\r\n      Aldous Hux'
b'ley (1894-1963)\r\n      Bra'
b've New World (1932)\r\n    \r'
b'\n\r\n    \r\n\r\n      \r\n\r\n     '
b'   Chapter One\r\n      \r\n\r\n'


In [15]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [16]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [17]:
dataset = sequences.map(split_input_target)

In [18]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\r\n    \r\n\r\n      Aldous Hu'
Target: b'\n    \r\n\r\n      Aldous Hux'


### Create training batches

In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 25), (64, 25)), types: (tf.int64, tf.int64)>

### Build The Model

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [21]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(rnn_units,
                                       return_sequences=True, 
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.lstm.get_initial_state(x)
            
        x, states, state_c = self.lstm(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, [states, state_c]
        else: 
            return x

In [22]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### Try the model

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 25, 84) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  21504     
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
dense (Dense)                multiple                  86100     
Total params: 5,354,580
Trainable params: 5,354,580
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [26]:
sampled_indices

array([57, 81, 22, 23, 42, 32, 75, 70, 52, 80, 35, 41, 83, 81, 10, 45,  5,
       74, 50,  0, 75, 29, 78,  1, 42])

In [27]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'            For\r\n\r\n      '

Next Char Predictions:
 b'c\xc2\x8578NDupXzGM\xc2\xa7\xc2\x85+Q!tVuAx[UNK]N'


### Train the model

In [28]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [29]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 25, 84)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.430936


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [30]:
tf.exp(mean_loss).numpy()

84.01

In [31]:
model.compile(optimizer='adam', loss=loss)

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [33]:
EPOCHS = 10

In [34]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
228/228 [==============================] - 479s 2s/step - loss: 3.0649
Epoch 2/10
228/228 [==============================] - 698s 3s/step - loss: 2.0522
Epoch 3/10
228/228 [==============================] - 477s 2s/step - loss: 1.7820
Epoch 4/10
228/228 [==============================] - 473s 2s/step - loss: 1.5965
Epoch 5/10
228/228 [==============================] - 473s 2s/step - loss: 1.4757
Epoch 6/10
228/228 [==============================] - 470s 2s/step - loss: 1.3778
Epoch 7/10
228/228 [==============================] - 478s 2s/step - loss: 1.2931
Epoch 8/10
228/228 [==============================] - 473s 2s/step - loss: 1.2110
Epoch 9/10
228/228 [==============================] - 536s 2s/step - loss: 1.1262
Epoch 10/10
228/228 [==============================] - 604s 3s/step - loss: 1.0398


### Generate text

In [35]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices = skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())]) 
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits] 
        predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [36]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [37]:
start = time.time()
states = None
next_char = tf.constant(['Hello:'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

Hello: Bernard had been to his mise. She was get in a door in." He shooked. He knew weres the astonished. "But if I helitop!
      In intense by a sitting bleash racks, and smiles. No, behnerdthe seeds beft the window into queer, and the book, when he took resolved and silent, then any most petailed. Mustapha Mond. "He blood-disains it was still flughted to its three. Her voice begnaphed his hagarated coild his unger there there to cele, frop bad to mine its of the line, and a hearchies shrilk on being there and to the Blue and fifty tower, "I thought!"
      "But in the oxfers in the latches."
      Lenina suddedly, but no more things round the men and women. The citched was Delly to put eye. The gesures of the plane was just gratuits to the very lenstic o-covered awar of the science, smiling through the worting, bit even his eyes and subject sedious existence to starty-from eight men of the succeeding into a slice open. Nore's tall between though. "And she say this only sere!" he re

### Export the generator

In [38]:
tf.saved_model.save(one_step_model, 'one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [39]:
one_step_reloaded = tf.saved_model.load('one_step')

In [40]:
states = None
next_char = tf.constant(['Hello:'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Hello: I feel him for distratf wenting his chair. "Lenina," he said.
      "Nend your shoulders now they 
